In [268]:
import mysql.connector
import ast
from mysql.connector import Error
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
import itertools
from itertools import combinations
from itertools import permutations
from functools import reduce
import random
from collections import defaultdict
from graphconverter import SimpComplex

pw=""
db="dbtest"

In [269]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
def execute_queries_simp(connection, query, list_elem):
    cursor = connection.cursor()
    try:
        cursor.executemany(query, [(value,) for value in list_elem])
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")        

def execute_queries(connection, query, list_elem):
    cursor = connection.cursor()
    try:
        cursor.executemany(query, list_elem)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [270]:

query_nodes= """
SELECT nodes.ID_Nodes, Attributes.Name, Attributes.Value
FROM nodes
JOIN Att_Nodes
ON nodes.ID_Nodes = Att_Nodes.ID_Nodes
JOIN Attributes
ON Att_Nodes.ID_Att = Attributes.ID_Att
;
"""

query_edges= """
SELECT edges.sources, edges.targets
FROM edges
;
"""
# FETCH THE NODES WITH ATTRIBUTES

connection = create_db_connection("localhost", "emac", pw, db) # Connect to the Database
results=read_query(connection, query_nodes) # Read our defined query
print(results)
nodes=[]
seennodes=[]
for result in results:
  
  if result[0] not in seennodes:
    tuples=(result[0],{result[1]:result[2]})
    seennodes.append(result[0])
    nodes.append(tuples)
    
  else:
    for i in range(len(nodes)):
      if nodes[i][0]==result[0]:
        nodes[i][1][result[1]]=result[2]
           


# FETCH THE EDGES

results=read_query(connection, query_edges) # Read our defined query

edges=[]

for result in results:
  edges.append((result[0],result[1]))




MySQL Database connection successful
[('A', 'color', 'red'), ('E', 'color', 'red'), ('F', 'color', 'red'), ('B', 'color', 'blue'), ('C', 'color', 'green'), ('G', 'color', 'yellow'), ('D', 'shape', 'circle'), ('A', 'shape', 'triangle')]


In [271]:
SC=SimpComplex(nodes, edges)
nodes2=SC[0]
edges2=SC[1]
simplices=SC[2]
dico=SC[3]

vertices=[]
att_simp=[]

for i in range(len(nodes2)):
    vertices.append([nodes2[i][0]])
    
    
simp_strings = [str(sublist) for sublist in simplices]

v=[]

for i in range(len(vertices)):
    v.append((i,str(vertices[i])))


v_n=[]

for i in range(len(vertices)):
    for j in range(len(vertices[i])):
        for n in seennodes:
            if n in vertices[i][j]:
                v_n.append((n,i))
v_n
   
s=[]

for i in range(len(simp_strings)):
    s.append((i,simp_strings[i]))

s_v=[]

for i in range(len(simplices)):
  for k,ve in enumerate(vertices):
      
      if ve[0] in simplices[i]:
          
          s_v.append((k,i))



In [272]:
add_v = """
SELECT vertices.vertices,
simplices.simplices
FROM vertices
JOIN Simp_Vert
ON vertices.ID_V=Simp_Vert.ID_V 
JOIN simplices 
ON simplices.ID_S=Simp_Vert.ID_S;
"""
execute_query(connection, add_v)

Error: 'Unread result found'
